In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
!pip install python-dotenv openAI langchain_core langchain_openai

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [ ]:
# <호출(invoke) 출력>

# 라이브러리 불러오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 오픈AI의 대규모 언어 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿 정의: 주어진 주제에 대한 설명 요청
prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명을 해주세요")
# 출력 파서 정의: AI 메시지의 출력 내용을 추출
parser = StrOutputParser()
# 프롬프트, 모델, 출력 파서를 체인으로 연결
chain = prompt | model | parser

# 응답을 호출
chain.invoke({"topic":"더블딥"})


'더블딥(Double Dip)은 경제학에서 사용되는 용어로, 경기 침체가 두 번 발생하는 현상을 의미합니다. 보통 첫 번째 침체 후 경제가 회복되는 듯하지만, 다시 둔화되거나 침체에 빠지는 경우를 가리킵니다. 이 현상은 소비자 신뢰가 낮거나 불확실한 경제 상황에서 발생할 수 있으며, 경제 회복에 대한 불안감을 나타내기도 합니다. 더블딥은 금융 위기, 정책 실패, 외부 충격 등 여러 요인에 의해 촉발될 수 있습니다.'

In [ ]:
# <배치(Batch) 출력>

# 주어진 주제 리스트를 배치로 출력
chain.batch([{"topic": "더블딥"}, {"topic": "인플레이션"}])

['더블딥(두 번째 하강)은 경제학에서 경기침체의 형태 중 하나로, 첫 번째 경기침체에서 회복된 후 다시 경제가 침체되는 현상을 의미합니다. 이 과정은 통상적으로 경기 회복세가 일시적일 때 발생하며, 생산과 고용이 회복된 것처럼 보이지만 다시 악화되는 상황을 가리킵니다. 더블딥은 통화 정책, 투자 감소, 소비자 신뢰 부족 등 여러 요인에 의해 촉발될 수 있습니다. 이 현상은 경제 전반에 불확실성을 가져오고, 정책 입안자들에게는 복잡한 도전 과제가 됩니다.',
 '인플레이션은 일반적인 물가 수준이 지속적으로 상승하는 현상을 말합니다. 이는 화폐의 구매력이 감소함을 의미하며, 예를 들어 같은 금액으로 구매할 수 있는 상품의 수가 줄어드는 것입니다. 인플레이션의 원인은 다양한데, 수요가 공급을 초과할 때, 제조 비용이 증가할 때, 중앙은행의 통화 정책 등이 있습니다. moderate한 인플레이션은 경제 성장의 일부로 긍정적인 영향을 미칠 수 있지만, 지나치게 높은 인플레이션은 경제에 부정적 영향을 미쳐 소비자 신뢰와 투자에 악영향을 줄 수 있습니다.']

In [ ]:
# <스트림(Stream) 출력>

# 응답을 토큰 단위로 스트리밍하여 출력
for token in chain.stream({"topic":"더블딥"}):
    # 스트리밍된 내용을 출력, 각 내용을 붙여서 출력하며, 버퍼를 즉시 플러시하여 실시간으로 보여줌
    print(token, end="", flush=True)


더블딥(Dual Dip)은 경제학이나 금융에서 사용되는 용어로, 주로 경기 침체 상황을 설명하는 데 쓰입니다. 일반적으로 경제가 한 번 침체한 뒤 회복되다가 다시 또다시 침체에 빠지는 경우를 가리킵니다. 이는 'W' 형태의 회복 곡선으로 시각화될 수 있으며, 한 번의 침체 후 회복이 단기간에 끝나지 않고 다시 하락하는 현상을 나타냅니다. 주로 경제 위기나 불황의 연속적인 영향을 분석할 때 중요한 개념으로 활용됩니다.

In [ ]:
# <구성된 체인을 다른 러너블과 결합하기>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# "이 대답을 영어로 번역해 주세요"라는 질문을 생성하는 프롬프트 템플릿을 정의
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해 주세요: {answer}")

# 이전에 정의된 체인(chain)을 사용하여 대답을 생성하고, 그 대답을 영어로 번역하도록 프롬프트에 전달한 후, 모델을 통해 결과를 생성하여 최종적으로 문자열로 파싱하는 체인을 구성
composed_chain = {"answer": chain} | analysis_prompt | model | StrOutputParser()
# "더블딥"이라는 주제로 대답을 생성하고, 체인 실행
composed_chain.invoke({"topic": "더블딥"})

'"Double Dip" is a term used in economics to refer to a situation where a recession occurs followed by an additional period of recession. In other words, it describes a scenario where the economy appears to recover temporarily, only to return to a downward trend. This phenomenon can be triggered by instability in consumer confidence or various external factors, indicating that the economy worsens again before it fully recovers.'

In [ ]:
# <람다 함수를 사용한 체인을 통해 구성하기>

# 이전에 정의된 값들
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("{topic}에 대해 짧게 설명을 해주세요.")
chain = prompt | model | StrOutputParser()
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해 주세요: {answer}")

# 람다 함수를 사용한 체인 구성
composed_chain_with_lambda = (
    # 이전에 정의된 체인(chain)을 사용하여 입력된 데이터를 받아옵니다.
    chain
    # 입력된 데이터를 "answer" 키로 변환하는 람다 함수를 적용합니다.
    | (lambda input: {"answer": input})
    # "answer " 키를 가진 데이터를 영어로 번역하도록 프롬프트에 전달합니다.
    | analysis_prompt
    # 프롬프트에서 생성된 요청을 모델에 전달하여 결과를 생성합니다.
    | model
    # 모델에서 반환된 결과를 문자열로 파싱합니다.
    | StrOutputParser()
)
# "더블딥"라는 주제로 대답을 생성하고, 그 대답을 영어로 번역합니다.
composed_chain_with_lambda.invoke({"topic": "더블딥"})


'"Double Dip" is a term used in economics that generally refers to the phenomenon where an economy experiences a recession, shows a brief recovery, and then falls back into a recession again. This contrasts with V-shaped or U-shaped recoveries, as it indicates that the economy does not fully recover and instead exhibits repeated downturns. It typically occurs during serious economic crises or recessions and can happen in situations where consumer confidence and investment sentiment have not fully recovered.'

In [ ]:
# <`.pipe()`를 통해 체인 구성하기>

# (방법1) 여러 작업을 순차적으로 .pipe를 통해 연결하여 체인 구성하기
composed_chain_with_pipe = (
  # 이전에 정의된 체인(chain)으로 입력된 데이터를 받아옴
  chain
  # 입력된 데이터를 "answer" 키로 변환하는 람다 함수 적용
  .pipe(lambda input: {"answer": input})
  # analysis_prompt를 체인에 연결하여 설명을 영어로 번역하는 작업 추가
  .pipe(analysis_prompt)
  # 모델을 사용해 응답 생성
  .pipe(model)
  # 생성된 응답을 문자열로 파싱
  .pipe(StrOutputParser())
)
# "더블딥"이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})


'Double Dip refers to a situation in the economy where two recessions occur. It describes a scenario where the economy temporarily recovers after the first recession, only to quickly lead into a second recession. This phenomenon typically arises in situations where economic recovery is incomplete or consumer confidence is low, making sustained growth difficult. Economically, it represents a complex business cycle characterized by the repetition of downturns and recoveries.'

In [ ]:
# (방법2) 좀 더 간단하게 연결하기
composed_chain_with_pipe = chain.pipe(lambda input:{"answer":input}, analysis_prompt, model, StrOutputParser())

# "더블딥"이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})

'Double Dip is a term used in economics that refers to the phenomenon where the economy recovers after a recession, only to fall into another recession again. In other words, instead of a V-shaped recovery, the economic cycle appears in a W shape. This scenario can arise due to factors such as declining consumer confidence, job instability, and external shocks, and it typically reflects uncertainty and volatility in the economy.'

In [ ]:
# <`RunnableParallel`을 이용한 체인 구성>
from langchain_core.runnables import RunnableParallel
# OpenAI 모델 초기화
model = ChatOpenAI()
# 한국어 설명 생성 프롬프트 체인
kor_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧은 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 영어 설명 생성 프롬프트 체인
eng_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 영어로 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 병렬 실행을 위한 RunnableParallel 설정
parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
# 주제에 대한 한국어와 영어 설명 생성
result = parallel_chain.invoke({"topic": "더블딥"})
# 결과 출력
print("한글 설명:", result['kor'])
print("영어 설명:", result['eng'])


한글 설명: 더블딥(Double deep)은 딥러닝 모델 중 하나로, 심층신경망(deep neural network)의 확장형태입니다. 즉, 더블딥은 두 개의 딥러닝 모델을 연결한 것을 의미합니다. 이러한 구조를 사용하여 보다 복잡하고 정교한 문제를 해결하거나, 더 많은 데이터를 처리할 수 있습니다.더블딥은 일반적인 딥러닝 모델보다 더 강력한 성능을 보여줄 수 있지만, 그만큼 학습과 모델 구성에 더 많은 노력이 필요합니다.
영어 설명: Double deep is a type of artificial neural network with multiple layers of neurons, allowing for more complex and accurate patterns to be learned and processed.


In [ ]:
# <`RunnableParallel`를 이용한 체인 구성하기>
from langchain_core.runnables import RunnableParallel
# OpenAI 모델 초기화
model = ChatOpenAI()
# 한국어 설명 생성 프롬프트 체인
kor_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 영어 설명 생성 프롬프트 체인
eng_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 영어로 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 병렬 실행을 위한 RunnableParallel 설정
parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
# 주제에 대한 한국어와 영어 설명 생성
result = parallel_chain.invoke({"topic": "더블딥"})
# 결과 출력
print("한글 설명:", result['kor'])
print("영어 설명:", result['eng'])


한글 설명: 더블딥은 두 개 이상의 노래를 동시에 녹음하는 음악 기술이다. 각 노래는 다른 리듬, 템포, 가사 및 음악적 특징을 가지고 있으며, 오디오 편집 소프트웨어를 사용하여 이 노래들을 함께 조합하여 새로운 음악적 경험을 만들어낸다. 이는 혼합된 사운드와 효과를 만들어내는 창의적인 기술로, 음악 제작자와 프로듀서들 사이에서 널리 사용되고 있다.
영어 설명: Double deep learning is a machine learning method that combines two deep learning models to improve the performance and accuracy of the overall system. By combining the strengths of two separate models, it aims to maximize the potential of deep learning techniques for solving complex problems.


In [ ]:
# 참고
class CustomLCEL:
    def __init__(self, value):
        self.value = value  # 객체 생성 시 값을 초기화합니다.
    def __or__(self, other):
        if callable(other):
            return CustomLCEL(other(self.value))  # other가 함수일 경우, 함수를 호출하고 그 결과를 새로운 객체로 반환합니다.
        else:
            raise ValueError("Right operand must be callable")  # other가 함수가 아니면 에러를 발생시킵니다.
    def result(self):
        return self.value  # 현재 값을 반환합니다.
# 문자열 끝에 느낌표를 추가하는 함수
def add_exclamation(s):
    return s + "!"
# 문자열을 뒤집는 함수
def reverse_string(s):
    return s[::-1]
# 파이프라인을 생성하여 순차적으로 문자열 변환 작업을 수행합니다.
custom_chain = (
    CustomLCEL("랭체인 공부하기")  # "랭체인 공부하기"로 초기화된 객체 생성
    | add_exclamation  # 느낌표 추가
    | reverse_string  # 문자열 뒤집기
)
# 최종 결과를 출력합니다.
result = custom_chain.result()
print(result)
# 출력: !기하부공 인체랭


!기하부공 인체랭
